In [1]:
import open3d as o3d
import numpy as np

import copy

In [2]:
def creat_crystal(position, direction, H, R):
    # position にて長さ H, （外接円）半径 R, 向きが direction の正六角結晶を作る関数
    # positon = (p1, p2, p3)
    # direction = (d1, d2, d3)
    # direction は原点からのベクトル
    # 原点にて正六角結晶を作る --> 指定した向きに回転させる --> 指定した位置に平行移動させる --> return
    
    # 六角柱と2つのコーンを組み合わせる
    hex_cy = o3d.geometry.TriangleMesh.create_cylinder(radius=R, height=H, resolution=6, split=1)
    hex_con = o3d.geometry.TriangleMesh.create_cone(radius=R, height=1.0, resolution=6, split=1, create_uv_map=False)
    
    con_t = copy.deepcopy(hex_con).translate((0, 0, H/2))
    
    con_rev = copy.deepcopy(con_t)
    R = con_rev.get_rotation_matrix_from_xyz((0, np.pi, 0))
    con_rev.rotate(R, center=(0, 0, 0))
    
    crystal = hex_cy + con_t + con_rev
    
    # 回転
    R2 = crystal.get_rotation_matrix_from_xyz(direction)
    crystal.rotate(R2, center=(0, 0, 0))
    
    # 平行移動
    crystal2 = copy.deepcopy(crystal).translate(position)
    
    #陰をつける(?)
    crystal2.compute_vertex_normals()
    
    return crystal2


In [ ]:
def grow_crystal(var, num, h, hv, r, rv, Vvar):
    # 平面上に num*num 個の結晶を分散 var で成長させる関数
    # 結晶の長さは N(h, hv)の正規分布に従うとする
    # 結晶の半径は N(h, hv)の正規分布に従うとする
    # 結晶の向きの分散 --> Vvar
    
    # 平面上の点群 xyz をとる（後で creat_crystal関数の position になる
    x = np.linspace(0, 100, num)
    mesh_x, mesh_y = np.meshgrid(x, x)
    z = 0
    xyz = np.zeros((np.size(mesh_x), 3))
    xyz[:, 0] = np.reshape(mesh_x, -1)
    xyz[:, 1] = np.reshape(mesh_y, -1)
    xyz[:, 2] = np.reshape(0, -1)
    
    x_noise = np.random.normal(loc = 0, scale = var, size = num*num)
    y_noise = np.random.normal(loc = 0, scale = var, size = num*num)
    
    xyz[:, 0] = xyz[:, 0] + x_noise
    xyz[:, 1] = xyz[:, 1] + y_noise
    
    # 結晶の向きを表すベクトルの集合 Vxyz を作る
    Vxyz = np.zeros((np.size(mesh_x), 3))
    
    Vxyz[:, 0] = np.random.normal(loc = 0, scale = Vvar, size = num*num)
    Vxyz[:, 1] = np.random.normal(loc = 0, scale = Vvar, size = num*num)
    #Vxyz[:, 2] = abs(np.random.normal(loc = 0, scale = Vvar, size = num*num))
    
    # 結晶の集合 --> field
    field = list()
    
    for (p, d) in zip(xyz, Vxyz):
        height = np.random.normal(loc = h, scale = hv)
        radius = np.random.normal(loc = r, scale = rv)
        
        field.append(creat_crystal(p, d, height, radius))
    
    # 平面メッシュ
    face = o3d.geometry.TriangleMesh.create_box(width=200.0, height=200.0, depth=0.01, create_uv_map=False)
    face2 = copy.deepcopy(face).translate((-50, -50, 0))
    
    face2.paint_uniform_color([0.1, 0.1, 0.1])
    field.append(face2)
        
    return field

var = 5
num = 10
h = 10
hv = 2
r = 1.0
rv = 0
Vvar = 0.5
o3d.visualization.draw_geometries(grow_crystal(var, num, h, hv, r, rv, Vvar))